<a href="https://colab.research.google.com/github/ALANT535/ALANT535/blob/main/Signature_Forgery_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mounting Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Importing various libraries


In [ ]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow


#Reading data


In [ ]:

directory1="/content/gdrive/MyDrive/Signature_forgery/Dataset/full_forg"

print(os.listdir(directory1))



In [ ]:
#os.path.join works with normal strings

img_arr=cv2.imread(os.path.join(directory1,'forgeries_2_1.png'),0)
#0 is for grayscale

# cv2.imshow(img_arr)           this function doesn't work in google collabs
print(img_arr)                  #instead use cv2_imshow()
print(np.shape(img_arr))

cv2_imshow(img_arr)

In [ ]:
#deciding a size for standard image

directory="/content/gdrive/MyDrive/Signature_forgery/Dataset/full_forg"

for img in os.listdir(directory):
  img_arr = cv2.imread(os.path.join(directory,img))
  print(np.shape(img_arr))

Creating the training data

In [ ]:
directory="/content/gdrive/MyDrive/Signature_forgery/Dataset"
training_data=[]
img_len = 575
img_breadth = 250


def create_training_data():
  categories=["full_forg","full_org"]
  for category in categories:
    path=os.path.join(directory,category)           #/content/drive/MyDrive/Signature_forgery/Dataset/full_forg

    for img in os.listdir(path):

      img_array=cv2.imread(os.path.join(path,img),0)      #/content/drive/MyDrive/Signature_forgery/full_forg/Dataset/forgeries_2_11.png
      new_array=cv2.resize(img_array, ( img_len ,img_breadth ))
      label=categories.index(category)                   #0`

      training_data.append([new_array,label])


In [ ]:
create_training_data()

Checking sizes of all images

In [ ]:
len(training_data)

48

In [ ]:

for i in training_data:
  print(np.shape(i[0]))

In [ ]:
cv2_imshow(training_data[0][0])

Printing some images

In [ ]:
for i in training_data[:3]:
  cv2_imshow(i[0])

Shuffling the training data

In [ ]:
import random
random.shuffle(training_data)

Checking if shuffled or not

In [ ]:
for i in training_data[:10]:
  print(i[1])


1
0
0
0
1
1
1
0
0
1


Store the data using pickle and uploading it to google drive

In [ ]:
X=[]
y=[]

for features,label in training_data:
  X.append(features)
  y.append(label)

print(type(X))

<class 'list'>


In [ ]:
# cv2_imshow(X[0])      #checking difference between reshaping the
print(np.shape(X))

In [ ]:
X= np.array(X).reshape(-1,img_breadth,img_len,1)  #important to run this becae we can only feed a numpy array into a neural network it seems so
# cv2_imshow(X[0])      #checking difference between reshaping the
# print(np.shape(X))
# print(X)

Dumping the training data

In [ ]:
import pickle

Pickle_file_directory = '/content/gdrive/MyDrive/Signature_forgery/Pickle_files'

pickle_out=open(os.path.join(path,'X.pickle'),'wb')    #or alternatively we can also use path+'/'+'X.pickle'
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out=open(os.path.join(path,'y.pickle'),'wb')
pickle.dump(y,pickle_out)
pickle_out.close()




Loading the training data

In [ ]:
import pickle

Pickle_file_directory = '/content/gdrive/MyDrive/Signature_forgery/Pickle_files'

pickle_in=open(os.path.join(Pickle_file_directory,'X.pickle'),'rb')    #or alternatively we can also use path+'/'+'X.pickle'
X=pickle.load(pickle_in)
pickle_in.close()

pickle_in=open(os.path.join(Pickle_file_directory,'y.pickle'),'rb')
y=pickle.load(pickle_in)
pickle_in.close()




In [ ]:
y=np.array(y)

In [ ]:
np.shape(X)

(48, 250, 575, 1)

In [ ]:
np.shape(y)

(48,)

In [ ]:
y[:10]

So now the data has been loaded and saved into the google drive successfully


#Making the mdoel

In [ ]:
import tensorflow as tf
import keras.api._v2.keras as keras
from keras.models import Sequential
from keras.layers import  Dropout,Activation,Dense,Flatten,MaxPooling2D,Conv2D

              #used to have to do this because the previous import statement didnt work previously
# from keras import layers,models

In [ ]:
X_normalised = X/255.0      #normalise X data to lie in between 0 and 1

In [ ]:
np.shape(X_normalised)

(48, 250, 575, 1)

In [ ]:
model=Sequential()
model.add(Conv2D(64, (3,3),input_shape = X_normalised.shape[1:] ))
model.add(Activation("relu"))         #activation layer doesnt have to be seperate
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())     #need to flatten because Dense layer uses 1D layer while Conv2D and MaxPooling2D create 2D arrays
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))      #the activations options include relu, softmax, Tanh, Swish


In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])   #binary_cross_entropy because there are only two classes

In [ ]:
print(np.shape(X_normalised[0]))

In [ ]:
print(X_normalised[0])

In [ ]:
model.fit(X_normalised, y, epochs=25, validation_split=0.2)

In [ ]:
a=model.predict(X_normalised)

2/2 [==============================] - 6s 1s/step


In [ ]:
a

In [ ]:
len(y)

In [ ]:
y

In [ ]:
y[38:]      #actual y values

array([0, 0, 1, 1, 1, 0, 1, 0, 0, 0])

In [ ]:
a[38:]      #predicted y values

array([[0.09083798],
       [0.11053593],
       [0.52980655],
       [0.71623474],
       [0.9873188 ],
       [0.59958214],
       [0.90011406],
       [0.03077655],
       [0.7952841 ],
       [0.04165261]], dtype=float32)